In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import time

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
# Prepare dataset
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=0.1307,std=0.3081)])

train_set = torchvision.datasets.MNIST(root = "./dataset",train=True,download=False,transform=transform)  #Make download == True to download dataset
test_set = torchvision.datasets.MNIST(root ="./dataset",train=False,download=False,transform=transform)

train_loader = torch.utils.data.DataLoader(train_set,batch_size=32,shuffle=True,num_workers=4)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=32,shuffle=True,num_workers=4)

In [4]:
# Building neural net
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(28*28*1,500)
        self.fc2 = nn.Linear(500,10)
    
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

In [6]:
# Training nn

model = Net()
model.to(device) # To transfer to GPU if available
optimizer = torch.optim.Adam(model.parameters(),lr=3e-4)
calculate_loss = nn.CrossEntropyLoss()
total_time = 0

for epoch in range(10):
    t0 = time.time() #start timer
    model.train()

    for batch_idx, data_set in enumerate(train_loader):
        optimizer.zero_grad()  # Clear the old gradients from last step  
        data , labels = data_set
        data = data.to(device)
        labels = labels.to(device)
        data = data.view(-1,28*28)
        output = model(data)   #forward
        output.to(device)
        loss = calculate_loss(output,labels) #calculate loss
        loss.backward()
        optimizer.step()

        # program timing
        torch.cuda.synchronize() 
        t1 = time.time()
        eta = t1-t0
        total_time +=eta
        if batch_idx % 100 == 0:
            loss_data = loss.data
            #train_losses.append(loss_data)
            print(
                'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \t Elapsed time :{:.2f} s'.
                format(epoch, batch_idx * len(data), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss_data,(eta)))

print('Total time to train model :{} s'.format(total_time))

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.313573 	 Elapsed time :0.45 s
Train Epoch: 0 [3200/60000 (5%)]	Loss: 0.400994 	 Elapsed time :0.65 s
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.387404 	 Elapsed time :0.88 s
Train Epoch: 0 [9600/60000 (16%)]	Loss: 0.211027 	 Elapsed time :1.13 s
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.327820 	 Elapsed time :1.33 s
Train Epoch: 0 [16000/60000 (27%)]	Loss: 0.192007 	 Elapsed time :1.55 s
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.141380 	 Elapsed time :1.74 s
Train Epoch: 0 [22400/60000 (37%)]	Loss: 0.544283 	 Elapsed time :1.95 s
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.148879 	 Elapsed time :2.15 s
Train Epoch: 0 [28800/60000 (48%)]	Loss: 0.090868 	 Elapsed time :2.36 s
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.109127 	 Elapsed time :2.58 s
Train Epoch: 0 [35200/60000 (59%)]	Loss: 0.359023 	 Elapsed time :2.78 s
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.052550 	 Elapsed time :3.00 s
Train Epoch: 0 [41600/60000 (69%)]	Loss: 0.120808 	 Elapsed 

Total time to train model on GPU:40615.34852838516 s 
Total time to train model on CPU:62252.999192237854 s 

In [6]:
    # Predicting

    correct, total = 0 , 0
    predictions = []
    model.eval()

    for i, data_set in enumerate(test_loader):
        data, labels = data_set
        data, labels = data.to(device), labels.to(device)
        data = data.view(-1,28*28)
        output = model(data)
        _, predicted = torch.max(output.data,1)
        predictions.append(output)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('The testing set accuracy of the network is: %d %%' % (100 * correct / total))


The testing set accuracy of the network is: 79 %
